In [1]:
from __future__ import print_function

import logging
import os

import SimpleITK as sitk
import six
import numpy as np

import radiomics
from radiomics import featureextractor, getFeatureClasses
from radiomics import glcm
from tqdm import tqdm


In [14]:
radiomics.setVerbosity(logging.INFO)
radiomics.progressReporter = tqdm

In [15]:
kernel = 2
# size = 24

# imageName = sitk.GetImageFromArray(np.random.randn(size, size, size))
# # maskV = np.ones((size, size, size))
# # maskV[0][0][0] = 0
# maskName = sitk.GetImageFromArray(np.ones((size, size, size)))

In [8]:
imageSize = (3, 3, 3)

mask_data = np.zeros(imageSize, dtype=int)
mask_data[1,1,1] = 1
mask_data[2,2,2] = 1
mask = sitk.GetImageFromArray(mask_data)

image_data = np.random.randint(0, 100, size=imageSize)
image = sitk.GetImageFromArray(image_data)

In [13]:
extractor = featureextractor.RadiomicsFeatureExtractor()
extractor.settings

{'minimumROIDimensions': 2,
 'minimumROISize': None,
 'normalize': False,
 'normalizeScale': 1,
 'removeOutliers': None,
 'resampledPixelSpacing': None,
 'interpolator': 'sitkBSpline',
 'preCrop': False,
 'padDistance': 5,
 'distances': [1],
 'force2D': False,
 'force2Ddimension': 0,
 'resegmentRange': None,
 'label': 1,
 'additionalInfo': True}

In [18]:
features = extractor.execute(image, mask, voxelBased=True)
sitk.GetArrayFromImage(features['original_firstorder_Energy'])

d:\Desktop\workspace\todo\.venv\lib\site-packages\radiomics\firstorder.py:297: RuntimeWarning: Mean of empty slice
  return numpy.nanmean(numpy.absolute(percentileArray - numpy.nanmean(percentileArray, 1, keepdims=True)), 1)
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


array([[[   0.,    0.,    0.],
        [   0.,    0.,    0.],
        [   0.,    0.,    0.]],

       [[   0.,    0.,    0.],
        [   0., 7057.,    0.],
        [   0.,    0.,    0.]],

       [[   0.,    0.,    0.],
        [   0.,    0.,    0.],
        [   0.,    0., 7057.]]])

In [16]:
image_data**2

array([[[3136, 4225, 6889],
        [   0, 8100,    9],
        [1024, 2916, 2916]],

       [[1681,    9, 8649],
        [1089,    1,  676],
        [1089, 6724, 5625]],

       [[1764, 4624, 5184],
        [3844, 3136, 1521],
        [9409, 5041, 7056]]])

In [18]:
a_ext = glcm.RadiomicsGLCM(
        imageName, maskName,
        voxelBased=True, maskedKernel=False, voxelBatch=1024,
        **get_default_settings())
a = a_ext.execute()

batch: 100%|██████████| 12/12.0 [01:20<00:00,  6.74s/it]


In [19]:
a

{'JointEntropy': <SimpleITK.SimpleITK.Image; proxy of <Swig Object of type 'itk::simple::Image *' at 0x000001AE3E053D20> >}

In [20]:
featureMap = a['JointEntropy']
featureData = sitk.GetArrayFromImage(featureMap)

In [21]:
featureData.shape

(155, 240, 240)

In [22]:
featureData.shape

(155, 240, 240)

In [23]:
# featureData.
np.where(featureData == 4.53996538528106)

(array([61, 62, 62, 63, 72, 72, 73, 73, 75, 76, 78, 79, 79, 79, 84],
       dtype=int64),
 array([100,  73,  74,  84,  84,  85,  74,  75,  90,  87,  74,  74,  74,
         88,  92], dtype=int64),
 array([144, 137, 137, 132, 149, 152, 145, 145, 148, 131, 144, 141, 144,
        132, 139], dtype=int64))

In [24]:
featureData[61][100][144]

4.53996538528106

In [25]:
featureData.mean()

0.005851361887708959